In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
import tensorflow as tf

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
tf.__version__

'2.3.0'

In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [9]:
training_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/Deep Learning/xray/xray_dataset_covid19/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/gdrive/MyDrive/Deep Learning/xray/xray_dataset_covid19/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 153 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [10]:
from tensorflow.keras.layers import Conv2D

In [11]:
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [12]:
cnn.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [13]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the CNN on the Training set and evaluating it on the Test set
cnn.fit(x = training_set, validation_data = test_set, epochs = 30)

Epoch 1/30
5/5 [==============================] - 40s 8s/step - loss: 0.8045 - accuracy: 0.4837 - val_loss: 0.6749 - val_accuracy: 0.5000
Epoch 2/30
5/5 [==============================] - 5s 918ms/step - loss: 0.6903 - accuracy: 0.5359 - val_loss: 0.6133 - val_accuracy: 1.0000
Epoch 3/30
5/5 [==============================] - 5s 908ms/step - loss: 0.6391 - accuracy: 0.7582 - val_loss: 0.5395 - val_accuracy: 0.8250
Epoch 4/30
5/5 [==============================] - 5s 943ms/step - loss: 0.5965 - accuracy: 0.7255 - val_loss: 0.3848 - val_accuracy: 0.9750
Epoch 5/30
5/5 [==============================] - 4s 893ms/step - loss: 0.5084 - accuracy: 0.8039 - val_loss: 0.2299 - val_accuracy: 0.9750
Epoch 6/30
5/5 [==============================] - 5s 1s/step - loss: 0.4296 - accuracy: 0.8301 - val_loss: 0.1431 - val_accuracy: 0.9750
Epoch 7/30
5/5 [==============================] - 5s 941ms/step - loss: 0.4100 - accuracy: 0.8170 - val_loss: 0.1151 - val_accuracy: 0.9750
Epoch 8/30
5/5 [=========

In [14]:

# save it as a h5 file


from tensorflow.keras.models import load_model

cnn.save('covid.h5')
model = load_model('covid.h5')

In [27]:

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('/content/download.jpg', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)

In [28]:
result


array([[0.9497281]], dtype=float32)

In [29]:
if result[0][0]<=0.5:
    print("The person is normal")
else:
    print("The person has  pneumonia and traces of having covid are visible.")

The person has  pneumonia
